# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_csv = "../WeatherPy_done/output_data/weather_data.csv"
weather_df = pd.read_csv(weather_csv)

weather_df.head()

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Barrow,75,US,1587516496,78,71.29,-156.79,264.15,4.60
1,Mardin,98,TR,1587516496,47,37.31,40.74,287.69,2.79
2,New Norfolk,40,AU,1587516496,69,-42.78,147.06,285.93,1.79
3,Bluff,96,NZ,1587516496,84,-46.60,168.33,285.93,1.34
4,Khandyga,49,RU,1587516496,91,62.67,135.60,268.28,1.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# config
gmaps.configure(api_key=g_key)

# convert humidity to a float
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# store lat and long
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# make heatmap
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)
figure.add_layer(heat_layer)
figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# ideal weather
ideal_weather_df = weather_df

# ideal temps (had to do in Kelvin for some reason idk), ideal winds, and no clouds my dude
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 300) & (ideal_weather_df["Max Temp"] > 294)]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

ideal_weather_df

,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
30,Arraial do Cabo,0,BR,1587516500,76,-22.97,-42.03,296.60,2.35
88,Beni Mazar,0,EG,1587516507,17,28.50,30.80,298.78,3.05
206,Jardim,0,BR,1587516518,53,-21.48,-56.14,294.58,1.46
220,Flinders,0,AU,1587516519,35,-34.58,150.86,295.37,5.70
255,Assiut,0,EG,1587516522,16,27.18,31.18,298.06,2.66
277,Aswān,0,EG,1587516525,20,24.09,32.91,296.30,1.65
292,Kaduqli,0,SD,1587516526,14,11.02,29.72,298.02,3.16
341,Atar,0,MR,1587516531,18,20.52,-13.05,296.55,3.82
357,Newcastle,0,AU,1587516532,49,-32.93,151.78,297.59,7.70
365,Maham,0,IN,1587516533,34,28.98,76.30,297.09,1.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = ideal_weather_df

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# for look for hotel df
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    try:
        print(f"Closest hotel {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("City not found.....")
        
    print("------------")

KeyError: 'City'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map